In [ ]:
from ultralytics import YOLO
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os

# fine tune model

In [ ]:
#yolo settings datasets_dir='/Volumes/Tony SSD/Projekte/Studienarbeit/Datasets/Pith/tree-disk-pith.v2i.yolov12'

In [ ]:
base_model_path = "../models/yolo11s.pt"
dataset_path = "/Volumes/Tony SSD/Projekte/Studienarbeit/Datasets/Pith/tree-disk-pith.v2i.yolov12/data.yaml"

In [ ]:
# check if the dataset exists
if not os.path.exists(dataset_path):
    print("Dataset not found")
    exit()

# check if model exists
if not os.path.exists(base_model_path):
    print("Model not found")
    exit()

In [ ]:
# Load a model
base_model = YOLO(base_model_path)

# Train the model
train_results = base_model.train(
    data=dataset_path,  # path to dataset YAML
    epochs=100,  # number of training epochs
    imgsz=640,  # training image size
    device="cpu",  # device to run on, i.e. device=0 or device=0,1,2,3 or device=cpu
    batch=32, # batch size
)

In [ ]:
# Evaluate model performance on the validation set
metrics = base_model.val()

# results

## Plot segmented masks

In [ ]:
trained_model_path = "../models/yolo11s-det-pith.pt"

In [ ]:
# check if the model exists
if not os.path.exists(trained_model_path):
    print("Model not found")
    exit()

In [ ]:
trained_model = YOLO(trained_model_path)

In [ ]:
results = trained_model("../input/tree-disk4.jpg")

## Plot detect with original colors

In [ ]:
img = results[0].plot()

img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.title('Segmented objects')
plt.axis('off')
plt.imshow(img_rgb)

cv2.imwrite('detected_objects.jpg', img)